In [1]:
import importlib
import hashlib
from pathlib import Path
from benchmark_utils import render_distribution, visual_benchmark

# Directory with implementations
IMPLS_DIR = Path("impls")

# Cache: module_name -> {"hash": sha1, "result": benchmark_result}
benchmark_cache = {}

In [2]:
# Just restart the notebook and it will run all benchmarks again.
# Cached benchmarks are stored in memory and reused if the source file did not change.


from pprint import pprint


def compute_sha1(path):
    """Return SHA1 hash of the file's contents."""
    with open(path, "rb") as f:
        return hashlib.sha1(f.read()).hexdigest()


def load_module(name):
    """Import or reload impls.<name>."""
    full_name = "impls." + name

    if full_name in globals():
        module = importlib.reload(globals()[full_name])
    else:
        module = importlib.import_module(full_name)
        globals()[full_name] = module

    return module


def report(stats):
    render_distribution(stats)
    pprint(stats["metrics"])


# Iterate over implementation files
for file in IMPLS_DIR.glob("*.py"):
    name = file.stem

    # Skip interface and private modules
    if name == "_interface" or name.startswith("_"):
        continue

    print("Found implementation:", name)

    file_hash = compute_sha1(file)

    # If cached: check whether file changed
    if name in benchmark_cache:
        cached = benchmark_cache[name]
        cached_hash = cached["hash"]
        cached_stats = cached["stats"]

        if cached_hash == file_hash:
            print("  No changes. Using cached benchmark.")
            report(cached_stats)
            continue
        else:
            print("  File changed. Reloading and benchmarking again.")
    else:
        print("  First time seen. Benchmarking.")

    # Load/reload implementation
    module = load_module(name)

    # Require expected constructors
    if not hasattr(module, "producer_constructor") or not hasattr(
        module, "recoverer_constructor"
    ):
        print("  Missing constructors. Skipping.\n")
        continue

    N = 5
    D = 2**20
    producer_constructor = module.producer_constructor
    recoverer_constructor = module.recoverer_constructor
    D = getattr(module, "override_D", lambda _: D)(N)

    # Run benchmark
    stats = visual_benchmark(
        producer_constructor,
        recoverer_constructor,
        N=N,
        D=D,
        passes=10000,
    )

    # Store into cache
    benchmark_cache[name] = {
        "hash": file_hash,
        "stats": stats,
    }

    # Render the new result
    report(stats)

Found implementation: single_binary_fullrank
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:03<00:00, 3099.60it/s]


{'basic': {'expected_sample_burst_size': 5.500480820600491,
           'expected_sample_data_size': 4.0032410870107205,
           'expected_time_to_recover': 25.500999999999998,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.18237248993195682,
             'bit_redundancy': 104.25159567122584,
             'ideal_bit_efficency': 0.4329535469486706,
             'ideal_bit_redundancy': 30.45536995580514,
             'ideal_packet_efficiency': 2.164767734743353,
             'ideal_packet_redundancy': 6.091073991161027,
             'ideal_time_to_recover': 10.741754856915858,
             'ideal_transmitted_bits': 53.70877428457929,
             'packet_efficiency': 0.9118624496597841,
             'packet_redundancy': 20.85031913424517,
             'permeability': 0.4212287697312207,
             'saturation': 4.650680865754831,
             'transmitted_bits': 127.505}}
Found implementation: single_binary_matrix
  First 

benchmark: 100%|██████████| 10000/10000 [00:07<00:00, 1289.81it/s]


{'basic': {'expected_sample_burst_size': 5.513948266183012,
           'expected_sample_data_size': 3.9998272660534613,
           'expected_time_to_recover': 42.9364,
           'packet_size': 5,
           'payload_bits': 25.0},
 'derived': {'bit_efficency': 0.11645130937852266,
             'bit_redundancy': 189.682,
             'ideal_bit_efficency': 0.2769848656378154,
             'ideal_bit_redundancy': 65.25763895955936,
             'ideal_packet_efficiency': 1.3849243281890768,
             'ideal_packet_redundancy': 13.051527791911873,
             'ideal_time_to_recover': 18.051527791911873,
             'ideal_transmitted_bits': 90.25763895955936,
             'packet_efficiency': 0.5822565468926133,
             'packet_redundancy': 37.9364,
             'permeability': 0.42042480953018585,
             'saturation': 5.0,
             'transmitted_bits': 214.682}}
Found implementation: single_primitive_binary_matrix
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:01<00:00, 5481.37it/s]


{'basic': {'expected_sample_burst_size': 5.489252543122512,
           'expected_sample_data_size': 3.996284829721362,
           'expected_time_to_recover': 9.203299999999999,
           'packet_size': 5,
           'payload_bits': 2.584962500721156},
 'derived': {'bit_efficency': 0.0561746873560822,
             'bit_redundancy': 43.43153749927884,
             'ideal_bit_efficency': 0.1333356152096622,
             'ideal_bit_redundancy': 16.801924466099067,
             'ideal_packet_efficiency': 0.666678076048311,
             'ideal_packet_redundancy': 3.3603848932198126,
             'ideal_time_to_recover': 3.877377393364044,
             'ideal_transmitted_bits': 19.38688696682022,
             'packet_efficiency': 0.280873436780411,
             'packet_redundancy': 8.686307499855767,
             'permeability': 0.42130294496148607,
             'saturation': 0.5169925001442313,
             'transmitted_bits': 46.01649999999999}}
